
Podczas tych zajęć skupimy się na wyjaśnialności modelu. Często w praktyce głównym celem nie jest zbudowanie dobrego modelu uczenia maszynowego. To co jest potrzebne to zrozumienie danych. Fabryka chce wiedzieć dlaczego w produkcie pojawia się defekt a nie tylko być w stanie go przewidzieć. Trener piłkarski chce wiedzieć które czynniki zwiększają szansę na zdobycie gola, a nie tylko jaka jest szansa celnego strzału w danej sytuacji. Do tego w wielu przypadkach w których chodzi głównie o samą predykcję wyjaśnialność jest również bardzo mile widzianym dodatkiem. Często dobry wykres jest wart więcej niż zaawansowany model uczenia maszynowego

In [ ]:
import pandas as pd
import numpy as np
import dalex as dx
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

%matplotlib inline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz', compression='gzip', nrows=20640)
df.rename({'housing.csv': 'longitude'}, inplace=True, axis=1)
df

## Analiza danych

Pierwszym krokiem powinna być analiza danych (EDA - Expolratory Data Analysis). W tym celu można posługiwać się zarówno różnego rodzaju statystykami jak i grafikami

In [ ]:
df.isna().mean(0)

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(15,15))

In [ ]:
df.corr()

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(25,25))
plt.show()

In [ ]:
plt.scatter(df.longitude, df.latitude)

Przy tego typu wykresach punkty często nachodzą na siebie. Aby uzykać pełniejszy obraz warto skorzystać z parametru alpha

In [ ]:
plt.scatter(df.longitude, df.latitude, alpha=.1)

Warto też wykorzystać dodatkową przestrzeń na przekazanie informacji niż tylko wymiar x i y. Możemy zaingerować w rozmiar punktu czy jego odcień

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
s = ax.scatter(df.longitude, df.latitude, c=df.median_house_value, s=df.population/20, label='population', alpha=.05)
cb = fig.colorbar(s)
cb.solids.set(alpha=1)
fig.legend()
fig.show()

## Przetwarzanie danych

In [ ]:
df

Warto zacząć od podziału danych na zbiór treningowy i testowy aby uniknąć potencjalnego przecieku danych

In [ ]:
X = df.drop('median_house_value', axis=1)
Y = df['median_house_value']
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=.2)
trainX.shape, testX.shape

W celu przetwarzania danych dobrze jest wykorzystać pipeliny sklearnowe. Dzięki spójnemu API można połączyć różne techniki przetwarzania danych w jeden obiekt, którym łatwiej później zarządzać oraz go wykorzystywać

In [ ]:
imputer = SimpleImputer()
imputer.fit_transform(df[['total_bedrooms']])

Można też definiować swoje transformatory, które są zgodne z api sklearna, dzięki czemu mogą być wykorzystywane w analogiczny sposób i łączone

In [ ]:
class PerHouseholdsAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        return X.assign(roomsPerHouseholds=X.total_rooms/X.households,
                        bedroomsPerHouseholds=X.total_bedrooms/X.households,
                        populationPerHouseholds=X.population/X.households)

attr_adder = PerHouseholdsAttributesAdder()
attr_adder.transform(df)

In [ ]:
numCols = list(trainX.columns)
numCols.remove('ocean_proximity')

Poza klasycznym pipelinem, gdzie wyjście z poprzedniego kroku jest przekazywane do następnego można skorzystać również z ColumnTransformera, gdzie określamy które kolumny mają zostać użyte. Pipeliny można dowolnie zagnieżdżać - jeden pipeline może być elementem drugiego

In [ ]:
numTransformer = Pipeline([
    ('imputer', SimpleImputer().set_output(transform="pandas")),
    ('addColumns', PerHouseholdsAttributesAdder())
])

preprocessing = ColumnTransformer([
        ("numerical", numTransformer, numCols),
        ("ohe", OneHotEncoder(), ['ocean_proximity']),
    ])

preprocessing.fit_transform(trainX)

In [ ]:
preprocessing.transform(testX)

#### Zad 
Wytrenuj adekwatny model i zwaliduj jego jakość

#### Zad
Dodaj model jako ostatni element pipelinu

#### Zad
Na takim pipelinie można przeprowadzić teraz optymalizację hiperparametrów. Postaraj się uzyskać jak najlepszy wyniki. Zaingeruj nie tylko w model ale też w sam preprocessing

In [ ]:
fullPipeline.get_params().keys()

In [ ]:
param_dist = {
          'model__n_estimators': np.arange(10,30), 
          'preprocessing__numerical__imputer__strategy': ['mean', 'median'],
}

## Wyjaśnialność

Dalex to pakiet pythonowy służący do wyjaśnialności modeli. Użyjemy go, żeby zrozumieć lepiej dane i model. Na początku musimy wytworzyć obiekt Explainer

In [ ]:
exp = dx.Explainer(res.best_estimator_, trainX, trainY)

In [ ]:
fi = exp.model_parts()

Przyjrzyjmy się najpierw istotności poszczególnych atrybutów. To dość podstawowa technika badająca globalny wpływ danej cechy na rezultat. Podstawowe podejście zakłada sprawdzenie na ile podorszy się jakość wyników po usunięciu danej cechy.

In [ ]:
fi.plot()

Kolejna istotna technika to partial dependency plot. Dla danej cechy badamy jak wygląda średni wynik działania modelu dla poszczególnych wartości

In [ ]:
exp.model_profile().plot()

Podobne wykresy wygenerować można dla pojedyńczego wiersza. Tutaj dla każdej cechy ukazujemy jak zmieni się wynik w zależności od wartości na niej przy założeniu, że wszystkie pozostałe pozostają bez zmian

In [ ]:
exp.predict_profile(trainX.iloc[[15,80]]).plot()

Wartości SHAP są odpowiednikiem wartości Shapley'a. Szacowany jest wpływ poszczególnych wartości poszczególnych kolumn na wynik predykcji. Odbywa się to poprzez próbkowanie i zastępowanie danej wartości losową z rozkładu występującego w danych i zmierzeniu wpływu na predykcję

In [ ]:
exp.predict_parts(trainX.iloc[15], type='shap').plot()

In [ ]:
exp.predict_parts(trainX.iloc[15], type='shap').plot()

Ponieważ wynik jest oparty na próbkowaniu nie uzyskamy dwóch identycznych wyników pod rzad

In [ ]:
exp.predict_parts(trainX.iloc[88], type='shap').plot()

In [ ]:
exp.predict_parts(trainX.iloc[88], type='break_down').plot()

In [ ]:
exp.predict_parts(trainX.iloc[88], type='shap').result

#### Zad
Wydziel 3 klasy - tanie, średnie i drogie. Następnie dla każdej z nich znajdź reprezentatywne przykłady i wyjaśnich ich predykcje.

Wyobraź sobie zadanie klasyfikacyjne dla klas pies i kot. Wtedy dobrym przykładem byłoby pokazanie na przykład 3 różnych ras psów. Analiza predykcji 5 labradorów nie wniesie zbyt wiele dla zrozumienia modelu.

Inne podejścia stosowane w wyjaśnialności:
 - LIME - przybliżanie modelu lokalnie modelem liniowym
 - Anchor - lokalne przybliżenie modelu systemem regułowym
 - Prototyp - wyjaśnienie predykcji przy użyciu podobnego przykładu z danych treningowych
 - Counterfactual Explanation - pokazanie zbliżonego przykładu o odmiennej predykcji aby ukazać co musi się zmienić w celu zmiany predykcji np. odrzucony wniosek kredytowy

# Zad

- wybierz dowolny zbiór danych
- przygotuj raport:
 - analiza danych, wizualizacja
 - pipeline przetwarzający dane
 - model predykcyjny
 - wyjaśnialność
 
Założenie jest takie, że raport będzie przeglądała osoba, która zamówiła analizę, nie zna się na uczeniu maszynowym, ale rozumie domenę danych 